<a href="https://colab.research.google.com/github/aboelela924/IMDB_reviews_sentiment_classification/blob/master/IMDB_sentiment_classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
! pip install -q kaggle

In [3]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ahmedaboelela","key":"03d60c1bb39658475c447099877abfea"}'}

In [4]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 97% 25.0M/25.7M [00:00<00:00, 29.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 31.2MB/s]


In [7]:
import os 
if not os.path.isdir("/content/data"):
    os.mkdir("/content/data")

In [8]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import numpy as np 
import matplotlib.pyplot as plt

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
import zipfile
with zipfile.ZipFile("/content/imdb-dataset-of-50k-movie-reviews.zip", "r") as ref:
    ref.extractall("/content/data",)

In [55]:
import pandas as pd
data_file_path = "/content/data/IMDB Dataset.csv"
df=pd.read_csv(data_file_path, sep=',',header=[0])

In [56]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [57]:
from collections import Counter
def get_all_unique_words(pandas_csv):
    all_words = []
    for i, row in pandas_csv.iterrows():
        row["review"] = row["review"].lower()
        all_words.extend(nltk.tokenize.word_tokenize(row["review"]))
    
    unique_words_counter = Counter(all_words)
    return unique_words_counter

In [58]:
unique_words = get_all_unique_words(df)

In [59]:
UNIQUE_WORDS_COUNT = len(unique_words)
print(UNIQUE_WORDS_COUNT)

168707


In [23]:
def get_word2number_number2word(uniqe_words_counter):
    word2number = {}
    for i, word in enumerate(uniqe_words_counter):
        word2number[word] = i+1

    number2word = {}
    for word in uniqe_words_counter:
        number2word[word2number[word]] = word

    return word2number, number2word

In [24]:
word2number, number2word = get_word2number_number2word(unique_words)

In [25]:
for  word in word2number:
    print(f"{word}: {word2number[word]}")
    if word2number[word] == 15:
        break

one: 1
of: 2
the: 3
other: 4
reviewers: 5
has: 6
mentioned: 7
that: 8
after: 9
watching: 10
just: 11
1: 12
oz: 13
episode: 14
you: 15


In [26]:
def convert_review(word_converter, text):
    sentence = []
    words = nltk.tokenize.word_tokenize(text)
    for word in words:
        sentence.append(word_converter[word])
    return np.asarray(sentence, dtype=np.float32)

In [27]:
def convert_label(label):
    return 1 if label == "positive" else 0

In [28]:
def sequence_padding(arr, padding=512):
    if (len(arr) > padding):
        arr = arr[:padding]
    elif (len(arr) < padding):
        padding_length = (padding - len(arr))
        zeros = np.zeros((padding_length)) 
        arr = np.append(zeros, arr, axis=0)

    return arr

In [52]:
def process_data(data_csv):
    for i, row in data_csv.iterrows():
        row["review"] = convert_review(word2number,row["review"])
        row["review"] = sequence_padding(row["review"], 128)
        row["sentiment"] = convert_label(row["sentiment"])

In [53]:
# df = df.apply(lambda x: convert_review(word2,x) if x.name == 'review' else x)
# df = df.apply(lambda x: sequence_padding(x) if x.name == 'review' else x)
# df = df.apply(lambda x: convert_label(x) if x.name == 'sentiment' else x)

In [60]:
process_data(df)

In [61]:
df.head()

,review,sentiment
0,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",1
1,"[56.0, 201.0, 202.0, 203.0, 19.0, 32.0, 33.0, ...",1
2,"[126.0, 286.0, 25.0, 41.0, 56.0, 201.0, 287.0,...",1
3,"[364.0, 365.0, 251.0, 56.0, 366.0, 92.0, 56.0,...",0
4,"[415.0, 416.0, 251.0, 224.0, 417.0, 50.0, 3.0,...",1


In [62]:
for i, row in df.iterrows():
    print(len(row["review"]))
    break

128


In [63]:
target = df.pop("sentiment")

In [64]:
target.head()

0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: object

In [65]:
df.head()

,review
0,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."
1,"[56.0, 201.0, 202.0, 203.0, 19.0, 32.0, 33.0, ..."
2,"[126.0, 286.0, 25.0, 41.0, 56.0, 201.0, 287.0,..."
3,"[364.0, 365.0, 251.0, 56.0, 366.0, 92.0, 56.0,..."
4,"[415.0, 416.0, 251.0, 224.0, 417.0, 50.0, 3.0,..."


In [66]:
data = df["review"].to_numpy()
labels = target.to_numpy()

In [67]:
print(labels[0])

1


In [68]:
labels = labels.astype(np.uint8)
print(labels)

[1 1 1 ... 0 0 0]


In [69]:
review_length = 128

reviews = np.empty((len(data), review_length))
for i,row in enumerate(data):
    for j,item in enumerate(row):
        reviews[i][j] = item



In [70]:
print(len(reviews))

50000


In [71]:
TRAIN_LENGTH = 39999
VAL_LENGTH = 44999
TEST_LENGTH = 49999

In [72]:
print(reviews.shape)

(50000, 128)


In [73]:
reviews_train  = reviews[:TRAIN_LENGTH][:]
reviews_val = reviews[TRAIN_LENGTH:VAL_LENGTH][:]
reviews_test = reviews[VAL_LENGTH:TEST_LENGTH][:]


labels_train  = labels[:TRAIN_LENGTH]
labels_val = labels[TRAIN_LENGTH:VAL_LENGTH]
labels_test = labels[VAL_LENGTH:TEST_LENGTH]


In [80]:
ds_train = tf.data.Dataset.from_tensor_slices((reviews_train, labels_train))
ds_val = tf.data.Dataset.from_tensor_slices((reviews_val, labels_val))
ds_test = tf.data.Dataset.from_tensor_slices((reviews_test, labels_test))

In [81]:
ds_train = ds_train.shuffle(1000).padded_batch(10)
ds_val = ds_val.shuffle(1000).padded_batch(10)
ds_test = ds_test.shuffle(1000).padded_batch(10)

In [74]:
reviews_train_tensor = tf.convert_to_tensor(reviews_train)
labels_train_tensor = tf.convert_to_tensor(labels_train)

reviews_val_tensor = tf.convert_to_tensor(reviews_val)
labels_val_tensor = tf.convert_to_tensor(labels_val)

reviews_test_tensor = tf.convert_to_tensor(reviews_test)
labels_test_tensor = tf.convert_to_tensor(labels_test)

In [75]:
print(labels_train_tensor.shape)

(39999,)


In [77]:

# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(UNIQUE_WORDS_COUNT, 1024),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024)),
#     tf.keras.layers.Dense(1024, activation='relu'),
#     tf.keras.layers.Dense(1)
# ])

# model.summary()
vocab_size = len(word2number) + 1 
output_size = 1
embedding_dim = 256
hidden_dim = 128
model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    layers.Bidirectional(tf.keras.layers.LSTM(hidden_dim)),
    layers.Dense(hidden_dim, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 256)         43189248  
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 512)         1050624   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               656384    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 44,929,281
Trainable params: 44,929,281
Non-trainable params: 0
__________________________________________

In [78]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(ds_train, epochs=10,
                    validation_data=ds_val,
                    validation_steps=30)

In [83]:
model.evaluate(ds_test)

500/500 [==============================] - 5s 11ms/step - loss: 0.5673 - accuracy: 0.8408


[0.5673018097877502, 0.8407999873161316]

In [93]:
labels = ["Negative", "Positive"]
postive_review_test = "The character build was good. You can follow their development through the movie."
negative_review_test = "The movie ws not that good. there is a lot of flaws in the story."
postive_review_test = sequence_padding(convert_review(word2number,postive_review_test.lower()), 10)
negative_review_test = sequence_padding(convert_review(word2number,negative_review_test.lower()), 10)
postive_review_test = postive_review_test[None,:]
negative_review_test = negative_review_test[None,:]

In [97]:
result1 = model.predict(postive_review_test)
result2 = model.predict(negative_review_test)
result1 = 0 if result1 < 0 else 1
result2 = 0 if result2 < 0 else 1
print(labels[result1])
print(labels[result2])

Positive
Negative
